In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,concatenate,Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
import re

In [2]:
df=pd.read_csv('training.csv',encoding='ISO-8859-1',header=None)
df

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
columns=df.columns
columns

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [4]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.columns=['Label','Text']
df.head()

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [6]:
df['Label'].replace({4:1},inplace = True)
df

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [7]:
def clean_tweet(tweet):
    '''
    Function to clean tweet text by removing links, special characters using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\d+)|(\w+:\/\/\S+)", " ", tweet).split())

In [8]:
df['Text'] = df['Text'].apply(clean_tweet)
df

,Label,Text
0,0,Awww that s a bummer You shoulda got David Car...
1,0,is upset that he can t update his Facebook by ...
2,0,I dived many times for the ball Managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it s not behaving at all i m mad why am i h...
...,...,...
1599995,1,Just woke up Having no school is the best feel...
1599996,1,TheWDB com Very cool to hear old Walt interviews
1599997,1,Are you ready for your MoJo Makeover Ask me fo...
1599998,1,Happy th Birthday to my boo of alll time Tupac...


In [9]:
df.Label.value_counts()

1    800000
0    800000
Name: Label, dtype: int64

In [10]:
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in df.Text]

In [11]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [12]:
# from nltk.corpus import stopwords
# stoplist = stopwords.words('english')
# def removeStopWords(tokens): 
#     return [word for word in tokens if word not in stoplist]
# filtered_words = [removeStopWords(sen) for sen in lower_tokens]
df['Text_Final'] = [' '.join(sen) for sen in lower_tokens]
df['tokens'] = lower_tokens

In [13]:
pos = []
neg = []
for l in df.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
df['Pos']= pos
df['Neg']= neg

df = df[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]
df.head()

,Text_Final,tokens,Label,Pos,Neg
0,awww that s a bummer you shoulda got david car...,"[awww, that, s, a, bummer, you, shoulda, got, ...",0,0,1
1,is upset that he can t update his facebook by ...,"[is, upset, that, he, can, t, update, his, fac...",0,0,1
2,i dived many times for the ball managed to sav...,"[i, dived, many, times, for, the, ball, manage...",0,0,1
3,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...",0,0,1
4,no it s not behaving at all i m mad why am i h...,"[no, it, s, not, behaving, at, all, i, m, mad,...",0,0,1


In [14]:
data_train, data_test = train_test_split(df,test_size=0.30,random_state=0)

In [15]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

14633234 words total, with a vocabulary size of 219539
Max sentence length is 53


In [16]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

6265886 words total, with a vocabulary size of 131935
Max sentence length is 47


In [17]:
from gensim import models
# Download link
# https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'   
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [18]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [19]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [20]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [21]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 219539 unique tokens.


In [22]:
train_embedding_weights = np.zeros((len(train_word_index)+1,EMBEDDING_DIM))
for word,index in train_word_index.items():
     train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(219540, 300)


In [23]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [24]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [2,3,4,5,6]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs, axis=1)
    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [25]:
label_names = ['Pos', 'Neg']

In [26]:
y_train = data_train[label_names].values

In [27]:
x_train = train_cnn_data
y_tr = y_train

In [28]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1,EMBEDDING_DIM, len(list(label_names)))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      65862000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 200)      120200      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      180200      embedding[0][0]                  
______________________________________________________________________________________________

In [29]:
num_epochs = 3
batch_size = 32
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.3, shuffle=True, batch_size=batch_size)

Epoch 1/3
24500/24500 [==============================] - 1674s 68ms/step - loss: 0.4768 - acc: 0.7711 - val_loss: 0.4177 - val_acc: 0.8085
Epoch 2/3
24500/24500 [==============================] - 1697s 69ms/step - loss: 0.4086 - acc: 0.8144 - val_loss: 0.4036 - val_acc: 0.8164
Epoch 3/3
24500/24500 [==============================] - 1716s 70ms/step - loss: 0.3854 - acc: 0.8265 - val_loss: 0.4064 - val_acc: 0.8175


In [30]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
sum(data_test.Label==prediction_labels)/len(prediction_labels)

469/469 [==============================] - 219s 467ms/step


0.8172125

In [53]:
model.save('depx007.h5')

In [54]:
import joblib
joblib.dump(tokenizer,'model_tokenizer')

['model_tokenizer']

In [31]:
text='I am happy'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.9935799  0.00640973]]
[0]


In [32]:
text='I am not happy'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.08902258 0.9109727 ]]
[1]


In [33]:
text='I am sad'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.01168683 0.9883045 ]]
[1]


In [34]:
text='I am depressed'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.01526377 0.98471594]]
[1]


In [35]:
text='I am not depressed'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.0281308  0.97185785]]
[1]


In [36]:
text='I hate my life'

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.05708945 0.94290423]]
[1]


In [37]:
text="I disappear sometimes. It's my thing."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.37116742 0.62885225]]
[1]


In [38]:
text="Sadness is to cry and to feel. But it’s that cold absence of feeling—that really hollowed-out feeling."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.0110409 0.9889561]]
[1]


In [39]:
text="Success is not final; failure is not fatal: It is the courage to continue that counts"

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.88131535 0.11873573]]
[0]


In [40]:
text="Opportunities don't happen. You create them."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.9641641  0.03578565]]
[0]


In [41]:
text="Stop chasing the money and start chasing the passion."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.68796134 0.31208977]]
[0]


In [42]:
text="The broken will always be able to love harder than most because once you’ve been in the dark, you learn to appreciate everything that shines."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.73103523 0.2689943 ]]
[0]


In [43]:
text="In addition to my other numerous acquaintances, I have one more intimate confidant… My depression is the most faithful mistress I have known—no wonder, then, that I return the love."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.5780876  0.42195594]]
[0]


In [44]:
text="There are wounds that never show on the body that are deeper and more hurtful than anything that bleeds."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.19732514 0.80268157]]
[1]


In [45]:
text="I can't eat and I can't sleep. I'm not doing well in terms of being a functional human, you know?"

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.3931939 0.6068242]]
[1]


In [46]:
text="Noble deeds and hot baths are the best cures for depression."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.8311243  0.16892958]]
[0]


In [47]:
text="Noble deeds and hot baths are not the best cures for depression."

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.30628777 0.6937268 ]]
[1]


In [48]:
text="I love my life"

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.987365   0.01264215]]
[0]


In [49]:
text='I am happy'
text = word_tokenize(text) 
text = lower_token(text)
# text = removeStopWords(text)
text = ' '.join(text)
predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.9935799  0.00640973]]
[0]


In [50]:
text='I am not happy'
text = word_tokenize(text) 
text = lower_token(text)
# text = removeStopWords(text)
text = ' '.join(text)
predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.08902258 0.9109727 ]]
[1]


In [51]:
text="I can't eat and I can't sleep. I'm not doing well in terms of being a functional human, you know?"
text = word_tokenize(text) 
text = lower_token(text)
# text = removeStopWords(text)
text = ' '.join(text)
predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.43546697 0.5645627 ]]
[1]


In [52]:
text="I am not sad"

predict=model.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.02805674 0.9719398 ]]
[1]


In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,concatenate,Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
import re
from tensorflow.keras.models import load_model
from joblib import load

In [2]:
dmodel = load_model('depx007.h5')
MAX_SEQUENCE_LENGTH = 50
tokenizer = load('model_tokenizer')

In [3]:
text='I am happy'

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.9935799  0.00640973]]
[0]


In [4]:
text='I am not happy'

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.08902258 0.9109727 ]]
[1]


In [5]:
text='I am depressed'

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.01526377 0.98471594]]
[1]


In [6]:
text="In addition to my other numerous acquaintances, I have one more intimate confidant… My depression is the most faithful mistress I have known—no wonder, then, that I return the love."

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.5780876  0.42195594]]
[0]


In [7]:
text="There are wounds that never show on the body that are deeper and more hurtful than anything that bleeds."

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.19732514 0.80268157]]
[1]


In [8]:
text="Noble deeds and hot baths are not the best cures for depression."

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.30628777 0.6937268 ]]
[1]


In [9]:
text="Noble deeds and hot baths are the best cures for depression."

predict=dmodel.predict(pad_sequences(tokenizer.texts_to_sequences([text]),maxlen=MAX_SEQUENCE_LENGTH))
print(predict)
print(np.argmax(predict,axis=1))

[[0.8311243  0.16892958]]
[0]


In [10]:
import json

with open( 'dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )